# Payoff Lookback (fixed)

Min/Max observes + strike fixe (simplifie).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_lookback_fixed

try:
    close_spy = fetch_spy_history()
except Exception:
    close_spy = None

if close_spy is not None and not getattr(close_spy, "empty", True):
    spot_ref = float(close_spy.iloc[-1])
else:
    spot_ref = 100.0
    close_spy = pd.Series([spot_ref], index=pd.Index([pd.Timestamp.today()]), name="Close")

min_ref = spot_ref*0.9
max_ref = spot_ref*1.1
strike_ref = spot_ref


## Payoff interactif
Min, max, strike fixe, sigma, r, T, call/put.


In [ ]:
sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

min_slider = widgets.FloatSlider(value=min_ref, min=spot_ref*0.5, max=spot_ref*1.0, step=1.0, description='Min path')
max_slider = widgets.FloatSlider(value=max_ref, min=spot_ref*1.0, max=spot_ref*1.6, step=1.0, description='Max path')
strike_slider = widgets.FloatSlider(value=strike_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K')
call_put = widgets.Dropdown(options=[('Call','call'),('Put','put')], value='call', description='Type')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        mn, mx, k = min_slider.value, max_slider.value, strike_slider.value

        fig_ts, ax_ts = plt.subplots(figsize=(8, 3))
        ax_ts.plot(close_spy.index, close_spy.values, label='SPY close (1y)')
        ax_ts.axhline(mn, color='teal', linestyle=':', label=f'Min path = {mn:.2f}')
        ax_ts.axhline(mx, color='gray', linestyle='--', label=f'Max path = {mx:.2f}')
        ax_ts.axhline(k, color='firebrick', linestyle='-.', label=f'K = {k:.2f}')
        ax_ts.set_ylabel('Prix')
        ax_ts.set_title('Clotures SPY (lookback fixed)')
        ax_ts.legend(loc='best')
        fig_ts.autofmt_xdate()
        plt.show()
        plt.close(fig_ts)

        view_dyn = view_lookback_fixed(spot_ref, mn, mx, k, option_type=call_put.value, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(spot_ref, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff (constant)')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(spot_ref, color='gray', linestyle='--', label=f'S0 = {spot_ref:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Lookback fixed (simplifié)')
        plt.show()
        plt.close(fig)
        display(Markdown(f"Prime ~ {premium:.4f}

- Payoff @ S0 = {payoff_s0:.4f}
- P&L net = {pnl_s0:.4f}"))

for sl in (min_slider, max_slider, strike_slider, sigma_slider, r_slider, T_slider, call_put):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([min_slider, max_slider, strike_slider, sigma_slider, r_slider, T_slider, call_put, output]))
